# Evaluation

In [21]:
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
# 设置全局字体
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['mathtext.fontset'] = 'stix'  # 数学公式字体
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
os.chdir(os.path.dirname(os.path.abspath("__file__")))
# 数据路径配置 (示例结构)
gt_forest_paths = [
    '../../data/benchmark/L1W.laz',
    '../../data_CULS/benchmark/merged_points.las',
    '../../data_Mergepoints/benchmark/merged_points.las',
    '../../data_NIBIO/benchmark/merged_points.las',
    '../../data_RMIT/benchmark/merged_points.las',
    '../../data_SCION/benchmark/merged_points.las',
    '../../data_TUWIEN/benchmark/merged_points.las'
]

# 每组真实数据对应的两个预测结果路径 (示例)
pred_configs = [
    {'Treelearn': '../../data/results/full_forest_withoutmamba/evaluation/evaluation_results.pt',
     'Ours': '../../data/pipeline/L1W/results/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_CULS/pipeline/results/full_forest/evaluation/evaluation_results.pt',
     'Ours': '../../data_CULS/results/mamba_results/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_Mergepoints/results/full_forest_withoutmamba/evaluation/evaluation_results.pt',
     'Ours': '../../data_Mergepoints/pipeline/results/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_NIBIO/results/full_forest_withoutmamba/evaluation/evaluation_results.pt',
     'Ours': '../../data_NIBIO/results/results_mamba/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_RMIT/pipeline/results/full_forest/evaluation/evaluation_results.pt',
     'Ours': '../../data_RMIT/results/results_mamba/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_SCION/results/full_forest_withoutmamba/evaluation/evaluation_results.pt',
     'Ours': '../../data_SCION/pipeline/results/full_forest/evaluation/evaluation_results.pt'},
    {'Treelearn': '../../data_TUWIEN/results/full_forest_withoutmamba/evaluation/evaluation_results.pt',
     'Ours': '../../data_TUWIEN/results/results_mamba/full_forest/evaluation/evaluation_results.pt'},
    # 补充剩余5组配置...
]

In [82]:
# 可视化参数优化
DATASET_NAMES = ['TREELEARN', 'CULS', 'Merge-Forest', 'NIBIO', 'RMIT', 'SCION', 'TUWIEN']  # 自定义数据集名称
MODEL_NAMES = ['Treelearn', 'Ours']  # 模型名称
COLORS = ['#1f77b4', '#ff7f0e']  # 蓝色和橙色
LINE_STYLES = ['-', '--']
MARKERS = ['o', 's']
LINE_WIDTH = 2.5
MARKER_SIZE = 8

# 创建雷达图
def create_radar_chart(ax, angles, data, color, label, linestyle, marker):
    ax.plot(angles, data, color=color, linewidth=LINE_WIDTH, linestyle=linestyle, label=label, marker=marker, markersize=MARKER_SIZE)
    ax.fill(angles, data, color=color, alpha=0.1)

# 初始化雷达图
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': 'polar'}, dpi=1000)
plt.subplots_adjust(left=0.1, right=0.95, top=0.9, bottom=0.1)

# 计算角度
angles = np.linspace(0, 2 * np.pi, len(DATASET_NAMES), endpoint=False).tolist()
angles += angles[:1]  # 闭合雷达图

# 遍历每个预测模型
for model_idx, model_name in enumerate(MODEL_NAMES):
    # 存储每个数据集的Precision
    precision_values = []
    
    # 遍历每组数据
    for group_idx, (gt_path, pred_paths) in enumerate(zip(gt_forest_paths, pred_configs)):
        # 加载评估结果
        eval_results = torch.load(pred_paths[model_name])
        z_data = eval_results['segmentation_results']['z_partition']
        
        # 提取Precision数据并取均值
        precision = z_data.iloc[:, 2:12].mean(0).mean() * 100  # 取所有分段的均值
        precision_values.append(precision)
    
    # 闭合数据
    precision_values += precision_values[:1]
    
    # 绘制雷达图
    create_radar_chart(
        ax, angles, precision_values,
        color=COLORS[model_idx],
        label=model_name,
        linestyle=LINE_STYLES[model_idx],
        marker=MARKERS[model_idx]
    )

# 设置雷达图角度标签
ax.set_xticks(angles[:-1])
ax.set_xticklabels(
    DATASET_NAMES,
    fontsize=0,
    color='black',
    rotation_mode='anchor',  # 旋转模式
    ha='center',  # 水平对齐方式
    va='center'   # 垂直对齐方式
)

# 设置角度标签的旋转角度
for label, angle in zip(ax.get_xticklabels(), np.degrees(angles[:-1])):
    label.set_rotation(angle - 90)  # 沿切线方向旋转

# 设置角度标签位置
ax.tick_params(axis='x', colors='black', labelsize=20)
ax.tick_params(axis='y', colors='gray', labelsize=20)

# 设置雷达图网格样式
ax.grid(color='gray', linestyle='--', linewidth=1, alpha=1)

# 设置角度标签方向
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

# 设置半径范围
ax.set_ylim(70, 100)
ax.set_yticks(np.arange(70, 101, 10))
ax.set_yticklabels([f'{int(y)}%' for y in np.arange(70, 101, 10)], fontsize=20, color='gray')

# 添加图例
legend_elements = [
    Line2D([0], [0], color=COLORS[0], lw=LINE_WIDTH, label='Treelearn'),
    Line2D([0], [0], color=COLORS[1], lw=LINE_WIDTH, label='Ours')
]
ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=20, frameon=False)

# 保存输出
plt.savefig(
    '../../plt/precision_radar_chart.png',
    dpi=1000,
    bbox_inches='tight',
    metadata={'CreationDate': None}
)
plt.show()

ValueError: x and y must have same first dimension, but have shapes (8,) and (2,)